# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for E Coli

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
N_THREADS = 16
IDENTITY_THRESHOLD=70
TAX_ID = 559292
ORGANISM = "yeast"
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)



,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
Q08986,Amino-acid transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,SAM3 YPL274W,S-adenosylmethionine permease SAM3 (S-adenosyl...,2.A.3.10.15,559292,MDILKRGNESDKFTKIETESTTIPNDSDRSGSLIRRMKDSFKQSNL...
P38967,Amino-acid transport,Membrane;Transmembrane,Transport,TAT2 LTG3 SAB2 SCM2 TAP2 YOL020W,Tryptophan permease (Tryptophan amino acid tra...,2.A.3.10.8,559292,MTEDFISSVKRSNEELKERKSNFGFVEYKSKQLTSSSSHNSNSSHH...
P39003,Sugar transport,Membrane;Transmembrane,Transport,HXT6 YDR343C D9651.12,High-affinity hexose transporter HXT6,NaN,559292,MSQDAAIAEQTPVEHLSAVDSASHSVLSTPSNKAERDEIKAYGEGE...
P38085,Amino-acid transport,Membrane;Transmembrane,Transport,TAT1 TAP1 VAP1 YBR069C YBR0710,Valine/tyrosine/tryptophan amino-acid permease...,2.A.3.10.9,559292,MDDSVSFIAKEASPAQYSHSLHERTHSEKQKRDFTITEKQDEVSGQ...
...,...,...,...,...,...,...,...,...
P40004,Sugar transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,YEA4 YEL004W,UDP-N-acetylglucosamine transporter YEA4,2.A.7.10.5,559292,MWNSLKAFALVFGGCCSNVITFETLMSNETGSINNLITFCQFLFVT...
Q06328,Amino-acid transport,Membrane;Transmembrane,Transport,YPQ2 YDR352W,Probable vacuolar amino acid transporter YPQ2 ...,2.A.43.2.8,559292,MSCSNGIWPTVSNLCGSLSFFTSVISLFPQIIETYRDKSVDGLSPY...
Q12235,Amino-acid transport,Cell membrane;Endoplasmic reticulum;Membrane;T...,Transport,YCT1 YLL055W L0578,High affinity cysteine transporter,2.A.1.14.20,559292,MSKVDVKIGADSISSSDEILVPSRLADVTLAFMEENDAAVPEITPE...


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
Q08986,Amino-acid transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,SAM3 YPL274W,S-adenosylmethionine permease SAM3 (S-adenosyl...,2.A.3.10.15,559292,MDILKRGNESDKFTKIETESTTIPNDSDRSGSLIRRMKDSFKQSNL...
P38967,Amino-acid transport,Membrane;Transmembrane,Transport,TAT2 LTG3 SAB2 SCM2 TAP2 YOL020W,Tryptophan permease (Tryptophan amino acid tra...,2.A.3.10.8,559292,MTEDFISSVKRSNEELKERKSNFGFVEYKSKQLTSSSSHNSNSSHH...
P39003,Sugar transport,Membrane;Transmembrane,Transport,HXT6 YDR343C D9651.12,High-affinity hexose transporter HXT6,NaN,559292,MSQDAAIAEQTPVEHLSAVDSASHSVLSTPSNKAERDEIKAYGEGE...
P38085,Amino-acid transport,Membrane;Transmembrane,Transport,TAT1 TAP1 VAP1 YBR069C YBR0710,Valine/tyrosine/tryptophan amino-acid permease...,2.A.3.10.9,559292,MDDSVSFIAKEASPAQYSHSLHERTHSEKQKRDFTITEKQDEVSGQ...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Amino-acid transport    34
Sugar transport         17
Name: labels, dtype: int64

## AAC, PAAC

In [7]:
for composition_type in ["aac", "paac"]:
    calculate_composition_feature(
        input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
        output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_{composition_type}.tsv",
        feature_type=composition_type,
    )

## PSSM

In [8]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM, IDENTITY_THRESHOLD, uniref_cluster_threshold, psiblast_iterations
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )

## COEXP

In [9]:
# TODO: optimize parameters first

## Reading dataframes

In [10]:

df_aac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_aac.tsv",
    index_col=0,
)
df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [11]:
df_all = pd.concat(
    [
        df_aac,
        df_paac,
        df_pssm_50_1it.rename(columns=lambda c: "PSSM_" + c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: "PSSM_" + c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: "PSSM_" + c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: "PSSM_" + c + "_90_3"),
    ],
    axis=1,
)

df_all

,A,C,D,E,F,G,H,I,K,L,...,PSSM_VL_90_3,PSSM_VK_90_3,PSSM_VM_90_3,PSSM_VF_90_3,PSSM_VP_90_3,PSSM_VS_90_3,PSSM_VT_90_3,PSSM_VW_90_3,PSSM_VY_90_3,PSSM_VV_90_3
Uniprot,,,,,,,,,,,,,,,,,,,,,
Q08986,0.085179,0.025554,0.027257,0.047700,0.069847,0.078365,0.010221,0.091993,0.044293,0.090290,...,0.425047,0.345351,0.402277,0.584440,0.282732,0.351044,0.351044,0.605313,0.759013,0.387097
P38967,0.069257,0.023649,0.035473,0.035473,0.069257,0.089527,0.023649,0.103041,0.050676,0.096284,...,0.462547,0.419476,0.458801,0.580524,0.376404,0.419476,0.426966,0.599251,0.709738,0.436330
P38085,0.074313,0.022617,0.035541,0.046850,0.064620,0.069467,0.021002,0.075929,0.048465,0.096931,...,0.441296,0.354251,0.437247,0.714575,0.301619,0.374494,0.392713,0.611336,0.878543,0.425101
P15380,0.078150,0.025518,0.041467,0.035088,0.057416,0.092504,0.019139,0.087719,0.039872,0.087719,...,0.419483,0.339960,0.405567,0.632207,0.300199,0.369781,0.359841,0.606362,0.819085,0.391650
P38206,0.048780,0.012195,0.029617,0.041812,0.095819,0.045296,0.012195,0.099303,0.057491,0.141115,...,0.603916,0.510542,0.590361,0.671687,0.507530,0.554217,0.564759,0.637048,0.774096,0.578313
Q12300,0.074705,0.011796,0.040629,0.039318,0.045872,0.072084,0.017038,0.093054,0.034076,0.077326,...,0.430147,0.345588,0.415441,0.645221,0.338235,0.382353,0.398897,0.562500,0.849265,0.391544
Q12010,0.045455,0.016234,0.045455,0.038961,0.071429,0.055195,0.009740,0.097403,0.029221,0.123377,...,0.424658,0.373288,0.407534,0.578767,0.345890,0.414384,0.404110,0.647260,0.852740,0.424658
Q03697,0.047511,0.011312,0.033937,0.040724,0.072398,0.061086,0.015837,0.083710,0.047511,0.128959,...,0.571429,0.440729,0.550152,0.677812,0.398176,0.483283,0.510638,0.574468,0.817629,0.531915
Q04602,0.049479,0.022135,0.039062,0.040365,0.062500,0.052083,0.019531,0.078125,0.055990,0.121094,...,0.604555,0.559006,0.590062,0.627329,0.542443,0.627329,0.627329,0.552795,0.786749,0.606625


## Custom Transformer to try all parameters

New version: Ignores all additional features that don't start with "PSSM"

In [12]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        
        not_pssm_feature = ~np.char.startswith(self.feature_names, "PSSM")
        self.mask = np.bitwise_or(np.bitwise_and(has_uniref, has_iterations), not_pssm_feature)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [13]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [14]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [15]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Combined Features

### Stats, Plots

Only three of the top 30 features comes from PAAC, all others come from PSSM with Uniref50 and 3 iterations. This is also the best parameter PSSM dataset that was found in the PSSM notebook.

In [16]:
df_stats = get_feature_stats(df_all, df_labels)
df_stats.sort_values("corr_abs", ascending=False).head(30)

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff
PSSM_TD_50_1,-0.607314,0.381623,0.058003,-0.167687,0.607314,0.406287,0.332297,0.411542,0.325773,0.085769
PSSM_TD_50_3,-0.589574,0.421540,0.065200,-0.102834,0.589574,0.448454,0.367713,0.448032,0.378151,0.069880
PSSM_IE_50_1,0.585433,0.079730,0.037387,0.702550,0.585433,0.064405,0.110379,0.061132,0.105085,0.043953
PSSM_IN_50_3,0.566032,0.132413,0.054663,0.753108,0.566032,0.110750,0.175739,0.102702,0.170616,0.067914
GM,0.549328,0.001401,0.001704,0.552434,0.549328,0.000746,0.002712,0.000000,0.002959,0.002959
PSSM_IQ_90_3,0.548188,0.172548,0.040490,0.761226,0.548188,0.157007,0.203629,0.157648,0.204589,0.046941
PSSM_TG_50_1,-0.543809,0.428604,0.056050,-0.059155,0.543809,0.449945,0.385923,0.449820,0.394175,0.055646
PSSM_GI_50_3,-0.527722,0.901725,0.063124,0.437127,0.527722,0.925048,0.855079,0.930275,0.851626,0.078649
PSSM_TP_50_1,-0.524726,0.414334,0.055005,-0.055387,0.524726,0.434542,0.373918,0.435636,0.351293,0.084343
PSSM_TH_50_1,-0.520976,0.384240,0.060950,-0.075786,0.520976,0.406472,0.339776,0.408768,0.324455,0.084313


### Independent test set

In [17]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_all, df_labels, test_size=0.2)

### Model selection

Linear SVC could be a good choice to avoid overfitting.

In [18]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced", random_state=0),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced", random_state=0),
    LinearSVC(max_iter=1e6, random_state=0),
    SVC(),
    RandomForestClassifier(random_state=0),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(random_state=0),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0)
CV folds: [0.855 0.855 0.873 0.667 0.619]
Mean: 0.774
Std: 0.108
###  SVC(class_weight='balanced')
CV folds: [1.    1.    0.855 0.385 0.855]
Mean: 0.819
Std: 0.227
###  RandomForestClassifier(class_weight='balanced')
CV folds: [0.855 0.795 0.667 0.385 0.855]
Mean: 0.711
Std: 0.177
###  LinearSVC(max_iter=1000000.0)
CV folds: [0.855 0.855 0.873 0.667 0.619]
Mean: 0.774
Std: 0.108
###  SVC()
CV folds: [1.    1.    0.667 0.385 0.667]
Mean: 0.744
Std: 0.233
###  RandomForestClassifier()
CV folds: [0.855 0.795 0.855 0.385 0.667]
Mean: 0.711
Std: 0.177
###  GaussianNB()
CV folds: [0.667 0.795 0.855 0.619 0.385]
Mean: 0.664
Std: 0.163
###  KNeighborsClassifier()
CV folds: [1.    0.564 0.855 0.385 0.667]
Mean: 0.694
Std: 0.216
###  SGDClassifier()
CV folds: [0.733 0.733 1.    0.667 0.619]
Mean: 0.75
Std: 0.132


### Parameter tuning

In [19]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 0.1, 10],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc = gsearch.best_estimator_


{'linearsvc__C': 1, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 'all'}
0.8497546897546897


In [20]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        SVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [1, 10, 100],
        "svc__gamma": ["scale", 0.001, 0.0001, 0.00001],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 90, 'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001}
0.875151515151515


### Dimensionality reduction

In [21]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 22


In [22]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 0.001, 0.01, 0.1],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc_pca = gsearch.best_estimator_

{'linearsvc__C': 0.01, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pca__n_components': 0.91, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 'all'}
0.8788455988455988


In [23]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [0.1, 1, 10],
        "svc__gamma": ["scale", 0.01, 0.1, 1],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'pca__n_components': 0.94, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 50, 'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.1}
0.9127272727272727


### Validation

In [24]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.913+-0.175


,Amino,Sugar,Macro,Weighted
precision,0.700,1.00,0.850,0.809
recall,1.000,0.25,0.625,0.727
f1-score,0.824,0.40,0.612,0.670
support,7.000,4.00,11.000,11.000


,Amino,Sugar
Amino,7,0
Sugar,3,1


In [25]:
best_estimator = best_estimator_lsvc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.879+-0.122


,Amino,Sugar,Macro,Weighted
precision,0.700,1.00,0.850,0.809
recall,1.000,0.25,0.625,0.727
f1-score,0.824,0.40,0.612,0.670
support,7.000,4.00,11.000,11.000


,Amino,Sugar
Amino,7,0
Sugar,3,1


# Alternative

In [26]:
from sklearn.model_selection import LeaveOneOut

X = df_all.values
labels = df_labels.labels
y = np.where(labels == "Sugar transport", 1, 0)


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(PSSMSelector(feature_names=feature_names), StandardScaler(), SVC()),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pca__n_components": [0.8, 0.9],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(),
    ),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


0.7935222672064778
0.831365139348134
